In [12]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt

from wordcloud import WordCloud


In [13]:

fr_videos_raw= pd.read_csv('./FRvideos.csv', sep=',')



In [14]:
fr_videos_raw.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Ro6eob0LrCY,17.14.11,Malika LePen : Femme de Gauche - Trailer,Le Raptor Dissident,24,2017-11-13T17:32:55.000Z,"Raptor""|""Dissident""|""Expliquez""|""moi""|""cette""|...",212702,29282,1108,3817,https://i.ytimg.com/vi/Ro6eob0LrCY/default.jpg,False,False,False,Dimanche.\n18h30.\nSoyez présents pour la vidé...
1,Yo84eqYwP98,17.14.11,"LA PIRE PARTIE ft Le Rire Jaune, Pierre Croce,...",Le Labo,24,2017-11-12T15:00:02.000Z,[none],432721,14053,576,1161,https://i.ytimg.com/vi/Yo84eqYwP98/default.jpg,False,False,False,Le jeu de société: https://goo.gl/hhG1Ta\n\nGa...
2,ceqntSXE-10,17.14.11,DESSINS ANIMÉS FRANÇAIS VS RUSSES 2 - Daniil...,Daniil le Russe,23,2017-11-13T17:00:38.000Z,"cartoon""|""pokémon""|""école""|""ours""|""мультфильм",482153,76203,477,9580,https://i.ytimg.com/vi/ceqntSXE-10/default.jpg,False,False,False,Une nouvelle dose de dessins animés français e...
3,WuTFI5qftCE,17.14.11,PAPY GRENIER - METAL GEAR SOLID,Joueur Du Grenier,20,2017-11-12T17:00:02.000Z,"Papy grenier""|""Metal Gear Solid""|""PS1""|""Tirage...",925222,85016,550,4303,https://i.ytimg.com/vi/WuTFI5qftCE/default.jpg,False,False,False,"Nouvel ,épisode de Papy Grenier ! Ce mois-ci o..."
4,ee6OFs8TdEg,17.14.11,QUI SAUTERA LE PLUS HAUT ? (VÉLO SKATE ROLLER ...,Aurelien Fontenoy,17,2017-11-13T16:30:03.000Z,"vélo""|""vtt""|""bmx""|""freestyle""|""bike""|""mtb""|""di...",141695,8091,72,481,https://i.ytimg.com/vi/ee6OFs8TdEg/default.jpg,False,False,False,Sauts à plus de 4 mètres de haut dans un tramp...


In [15]:

fr_category_id = pd.read_json('./FR_category_id.json')


In [16]:
fr_category_id.head()

,etag,items,kind
0,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'snippet': {'title': 'Film & Animation', 'cha...",youtube#videoCategoryListResponse
1,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'snippet': {'title': 'Autos & Vehicles', 'cha...",youtube#videoCategoryListResponse
2,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'snippet': {'title': 'Music', 'channelId': 'U...",youtube#videoCategoryListResponse
3,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'snippet': {'title': 'Pets & Animals', 'chann...",youtube#videoCategoryListResponse
4,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'snippet': {'title': 'Sports', 'channelId': '...",youtube#videoCategoryListResponse


In [17]:
fr_category_id['items'].iloc[0]

{'etag': '"ld9biNPKjAjgjV7EZ4EKeEGrhao/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
 'id': '1',
 'kind': 'youtube#videoCategory',
 'snippet': {'assignable': True,
  'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'title': 'Film & Animation'}}

In [18]:
fr_category_id_dict = {'category_id':[key for key in fr_category_id['items'].keys()],
                       'category_title':[y['snippet']['title'] for x,y in fr_category_id['items'].items()]}
fr_category_id_dict.keys(), fr_category_id_dict.values()

(dict_keys(['category_title', 'category_id']),
 dict_values([['Film & Animation', 'Autos & Vehicles', 'Music', 'Pets & Animals', 'Sports', 'Short Movies', 'Travel & Events', 'Gaming', 'Videoblogging', 'People & Blogs', 'Comedy', 'Entertainment', 'News & Politics', 'Howto & Style', 'Education', 'Science & Technology', 'Movies', 'Anime/Animation', 'Action/Adventure', 'Classics', 'Comedy', 'Documentary', 'Drama', 'Family', 'Foreign', 'Horror', 'Sci-Fi/Fantasy', 'Thriller', 'Shorts', 'Shows', 'Trailers'], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]]))

In [19]:
fr_category_id_df = pd.DataFrame.from_dict(fr_category_id_dict)

fr_videos = fr_videos_raw.merge(fr_category_id_df, how='inner', on='category_id').drop(columns='category_id')
fr_videos.loc[:5, ['title', 'category_title']]

,title,category_title
0,Malika LePen : Femme de Gauche - Trailer,Foreign
1,"LA PIRE PARTIE ft Le Rire Jaune, Pierre Croce,...",Foreign
2,De retour dans le Manoir hanté avec le Grand J...,Foreign
3,"T'es qui toi ? Squeezie, le youtubeur aux 4 mi...",Foreign
4,ON VOUS DÉVOILE NOTRE VRAI SALAIRE,Foreign
5,Jérémy Ferrari - On n'est pas couché 11 novemb...,Foreign


In [20]:
fr_videos.isna().sum()

video_id                     0
trending_date                0
title                        0
channel_title                0
publish_time                 0
tags                         0
views                        0
likes                        0
dislikes                     0
comment_count                0
thumbnail_link               0
comments_disabled            0
ratings_disabled             0
video_error_or_removed       0
description               2912
category_title               0
dtype: int64

In [21]:
fr_videos.loc[:5, ['video_id', 'trending_date', 'publish_time']]


,video_id,trending_date,publish_time
0,Ro6eob0LrCY,17.14.11,2017-11-13T17:32:55.000Z
1,Yo84eqYwP98,17.14.11,2017-11-12T15:00:02.000Z
2,nduL7G_gJoY,17.14.11,2017-11-12T19:00:08.000Z
3,QmpWE_SODZA,17.14.11,2017-11-13T17:30:01.000Z
4,GBVxEpQr8R8,17.14.11,2017-11-12T08:59:25.000Z
5,LhjhGOBIoHM,17.14.11,2017-11-12T00:53:02.000Z


In [23]:
fr_videos['trending_date'] = pd.to_datetime(fr_videos['trending_date'], format='%y.%d.%m')
fr_videos['publish_time'] = pd.to_datetime(fr_videos['publish_time'], format='%Y-%m-%d')

fr_videos['trending_date'] = pd.to_datetime(fr_videos['trending_date'].astype(str) + ' ' + pd.Series(['23:59:59+00:00']*fr_videos.shape[0]),
                                            format='%Y-%m-%d %H:%M:%S')


fr_videos['trending_time'] = pd.to_timedelta(fr_videos['trending_date'] - fr_videos['publish_time']).apply(lambda x: int(x.total_seconds()))

try:
    if (fr_videos['trending_time'] < 0).any():
        raise ValueError
except ValueError:
    print("Negative timedelta found ! You should have a look.")

In [24]:
(fr_videos['trending_time']//3600).describe()

count    40623.000000
mean        75.554735
std       1104.265542
min          3.000000
25%         30.000000
50%         35.000000
75%         52.000000
max      76564.000000
Name: trending_time, dtype: float64

In [25]:
for quantile, trd_time in fr_videos['trending_time'].quantile([0.80, 0.85, 0.90, 0.95, 0.97, 0.99]).iteritems():
    print("{}% of videos become trending in less than {} hours".format(int(quantile*100), int(trd_time//(3600))))

80% of videos become trending in less than 55 hours
85% of videos become trending in less than 56 hours
90% of videos become trending in less than 63 hours
95% of videos become trending in less than 80 hours
97% of videos become trending in less than 90 hours
99% of videos become trending in less than 156 hours


In [26]:
any_disabled = pd.Series([True if any([com, rat, err]) else False for com, rat, err in zip(fr_videos['comments_disabled'],
                                         fr_videos['ratings_disabled'],
                                         fr_videos['video_error_or_removed'])])


In [27]:
try:
    assert fr_videos[(fr_videos['comments_disabled'] == False) & 
          (fr_videos['ratings_disabled'] == False) & 
          (fr_videos['video_error_or_removed'] == False)].shape[0] == (any_disabled == False).sum()
    fr_videos['any_disabled'] = any_disabled
except AssertionError:
    print("any_disabled was not successfully computed !")

In [28]:
fr_videos['description'].head()

0    Dimanche.\n18h30.\nSoyez présents pour la vidé...
1    Le jeu de société: https://goo.gl/hhG1Ta\n\nGa...
2    Bonsoir à tous, \nJe tenais beaucoup à retourn...
3    Dans Salut Les Terriens, Thierry Ardisson, reç...
4    Nouvelle vidéo tous les dimanches matins 10h. ...
Name: description, dtype: object

In [29]:
fr_videos['description_length'] = fr_videos['description'].str.len()

fr_videos['description_length'] = fr_videos['description_length'].fillna(0).astype(int)

In [30]:


fr_videos['tags'].head()



0    Raptor"|"Dissident"|"Expliquez"|"moi"|"cette"|...
1                                               [none]
2    fantome"|"esprits"|"spiritisme"|"hanté"|"ouija...
3    salut les terriens"|"squeezie"|"jeux videos"|"...
4    mcfly"|"carlito"|"golden moustache"|"fatshow"|...
Name: tags, dtype: object

In [31]:
split_tags = fr_videos['tags'].str.replace('"', '').str.lower().str.split('|')
split_tags.head()

0    [raptor, dissident, expliquez, moi, cette, merde]
1                                             [[none]]
2    [fantome, esprits, spiritisme, hanté, ouija, m...
3    [salut les terriens, squeezie, jeux videos, in...
4    [mcfly, carlito, golden moustache, fatshow, ba...
Name: tags, dtype: object

In [32]:
split_tags.iloc[1]

['[none]']

In [33]:
print(split_tags.apply(lambda l: len(l) == 0).sum())


matchers = ['none','None', 'NONE']

nones = split_tags.apply(lambda l: tuple(s for s in l if any(xs in s for xs in matchers)))
nones.value_counts()

0


()                    35316
([none],)              5302
(manone enceinte,)        2
(master of none,)         1
(moytoul sa none,)        1
(annone,)                 1
Name: tags, dtype: int64

In [34]:
split_tags.apply(lambda l: l == ['[none]']).sum()

5302

In [35]:
split_tags_cleaned = split_tags.apply(lambda l: np.nan if l == ['[none]'] else l)

fr_videos['tags_count'] = split_tags_cleaned.apply(lambda x: int(len(x)) if type(x) == list else 0)

In [36]:
def input_n_tag(tags, n):
    try:
        n_tag = tags[n]
    except TypeError:
        n_tag = 'notag'

    except IndexError:
        n_tag = 'notag' 
    return n_tag 
    
fr_videos['tag1'] = split_tags_cleaned.apply(lambda l: input_n_tag(l, 0))
fr_videos['tag2'] = split_tags_cleaned.apply(lambda l: input_n_tag(l, 1))
fr_videos['tag3'] = split_tags_cleaned.apply(lambda l: input_n_tag(l, 2))
fr_videos['tag4'] = split_tags_cleaned.apply(lambda l: input_n_tag(l, 3))
fr_videos['tag5'] = split_tags_cleaned.apply(lambda l: input_n_tag(l, 4))

In [37]:


fr_videos.loc[:5, ['title', 'tags_count', 'tag1', 'tag2', 'tag5']]



,title,tags_count,tag1,tag2,tag5
0,Malika LePen : Femme de Gauche - Trailer,6,raptor,dissident,cette
1,"LA PIRE PARTIE ft Le Rire Jaune, Pierre Croce,...",0,notag,notag,notag
2,De retour dans le Manoir hanté avec le Grand J...,20,fantome,esprits,ouija
3,"T'es qui toi ? Squeezie, le youtubeur aux 4 mi...",8,salut les terriens,squeezie,ardisson
4,ON VOUS DÉVOILE NOTRE VRAI SALAIRE,12,mcfly,carlito,badass
5,Jérémy Ferrari - On n'est pas couché 11 novemb...,10,onpc,on n'est pas couche,christine angot


In [38]:
all_tags = split_tags_cleaned.explode().astype(str)
text = ', '.join(all_tags)

# Create wordcloud from single string
wordcloud = WordCloud().generate(text)
wordcloud = WordCloud(background_color="white", max_words=1000, max_font_size=40, relative_scaling=.5).generate(text)

plt.figure(figsize=(14, 10))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

AttributeError: 'Series' object has no attribute 'explode'

In [39]:
fr_videos['title_length'] = fr_videos['title'].str.len()


In [40]:


fr_videos.loc[:5, ['title', 'title_length']]



,title,title_length
0,Malika LePen : Femme de Gauche - Trailer,40
1,"LA PIRE PARTIE ft Le Rire Jaune, Pierre Croce,...",100
2,De retour dans le Manoir hanté avec le Grand J...,50
3,"T'es qui toi ? Squeezie, le youtubeur aux 4 mi...",82
4,ON VOUS DÉVOILE NOTRE VRAI SALAIRE,34
5,Jérémy Ferrari - On n'est pas couché 11 novemb...,59


In [41]:
print("Number of videos: {} for {} different channels.".format(fr_videos.shape[0], len(fr_videos['channel_title'].unique())))


Number of videos: 40623 for 6679 different channels.


In [42]:
fr_videos['channel_title'] = fr_videos['channel_title'].str.lower()


In [43]:
corr = fr_videos.loc[:, ['views', 'likes', 'dislikes', 'comments', 'trending_time', 'tags_count', 'description_length', 'title_length']].corr()

/home/glitch/anaconda3/envs/ten/lib/python3.5/site-packages/pandas/core/indexing.py:1472: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike



In [45]:
categories = fr_videos['category'].unique()
interactions_corr_list = [fr_videos[fr_videos['category'] == cat].loc[:, ['views', 'likes', 'dislikes', 'comments', 'trending_time', 'tags_count',
                                                                          'description_length', 'title_length']].corr() for cat in categories]

fig3 = go.Figure()

for idx, corr in enumerate(interactions_corr_list):
    if idx==0:
        fig3.add_trace(
            go.Heatmap(
                z=corr.values,
                x=corr.index,
                y=corr.index,
                colorscale="Earth",
                zmin=-1,
                zmax=1,
                visible=True))
    else:
         fig3.add_trace(
            go.Heatmap(
                z=corr.values,
                x=corr.index,
                y=corr.index,
                colorscale="Earth",
                zmin=-1,
                zmax=1,
                visible=False)) 

# Add buttons
fig3.update_layout(
    updatemenus=[
        go.layout.Updatemenu(
            active=0,
            x=0.8,
            y=1.2,
            buttons=list([
                dict(label=cat,
                     method="update",
                     
                     args=[{"visible": [False if sub_idx != idx else True for sub_idx, sub_cat in enumerate(categories)]},
                           {"title": "Correlation heatmap for category: " + cat}])
                for idx, cat in enumerate(categories)
            ] ) 
        )
    ])

KeyError: 'category'